In [ ]:
# Just the normal prequisites for using matplotlib and numpy in a Jupyter notebook
%matplotlib inline
# Use the svg backend, in my opinion it just makes better looking plots
%config InlineBackend.figure_format = 'svg'

import PySpice.Logging.Logging as Logging
logger = Logging.setup_logging( logging_level='CRITICAL')

import os
import sys
from pathlib import Path
import IPython
from scipy import signal
import scipy.fftpack
from PySpice.Unit import *
from PySpice.Spice.Parser import SpiceParser
from PySpice.Spice.Netlist import Circuit, SubCircuit, SubCircuitFactory
from PySpice.Spice.Library import SpiceLibrary
from PySpice.Probe.Plot import plot
from PySpice.Doc.ExampleTools import find_libraries
from PySpice.Math import *
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import unittest

directory_path = Path(os.path.abspath('')).resolve().parent.parent
spice_libraries_path = directory_path.joinpath("lib", "spice")
spice_library = SpiceLibrary(spice_libraries_path)

directory_path = Path(os.path.abspath('')).resolve()

## about
{: class="subtitle is-lowercase"}

Pegel is a voltage controlled amplifier (VCA). There are various designs to implement a VCA. rod elliott (ESP) has an article on VCA techniques [[1][1]]. diy synthesizer modules are usualy designed with an integrated VCA chip [[2][2]] or with a differential amplifier [[3][3]]. The integrated VCA chips are either obsolete or rather expensive. The differential amplifier can be built with transistors only. the design has some downsides. Even in the simulation the result is not symetryc and has a dc offset from the control voltage. This needs a lot of trimming to get an accurate result. but the biggest downside is, that this design can not do proper amplitude modulation (am). when the carrier signal goes below zero the base signal is completely muted. 

In [ ]:
fs = 10000
f1 = 8
f2 = 1

x1 = np.arange(fs)
y1 = np.sin(2 * np.pi * x1 * f1 / fs)

x2 = np.arange(fs)
y2 = np.sin(2 * np.pi * x2 * f2 / fs)

fig, (ax0, ax1) = plt.subplots(nrows=1, ncols=2, sharex=False, figsize=(18, 6))

#plot multiplication
ax0.plot(x1, y1, c='LightGrey')  # differential output
ax0.plot(x2, y2, c='LightGrey')  # differential output
ax0.plot(x2, y2 * y1, c='orange')  # differential output
ax0.grid()
ax0.set_xlabel('samples')
ax0.set_ylabel('amplitude')
ax0.set_title('multiplication', y=-0.2)

#plot multiplication with carrier added
ax1.plot(x1, y1, c='LightGrey')  # differential output
ax1.plot(x2, y2, c='LightGrey')  # differential output
ax1.plot(x2, y1 * y2 + y1, c='orange')  # differential output
ax1.grid()
ax1.set_xlabel('samples')
ax1.set_ylabel('amplitude')
ax1.set_title('add carrier to product', y=-0.2)
plt.show()


In [ ]:
fs = 1000 # sample rate 
f = 8 # the frequency of the signal
f2 = 2
x = np.arange(fs) # the points on the x axis for plotting
# compute the value (amplitude) of the sin wave at the for each sample
y = np.sin(10 * np.pi * f * (x/fs)) * np.sin(np.pi * f2 * (x/fs))

fig, (ax0, ax1) = plt.subplots(nrows=1, ncols=2, sharex=False, figsize=(18, 6))
ax0.plot(x, y)
ax0.set_xlabel('sample(n)')
ax0.set_ylabel('voltage(V)')
ax0.set_title('multiplication', y=-0.2)

y = (np.sin(10 * np.pi * f * (x/fs)) * np.sin(np.pi * f2 * (x/fs))) + np.sin(np.pi * f * (x/fs)) 
ax1.plot(x, y)
ax1.set_xlabel('sample(n)')
ax1.set_ylabel('voltage(V)')
ax1.set_title('add carrier to product', y=-0.2)


plt.show()

this is not real amplitude modulation. when the signal is negative the phase of the output is inverted. this can be adjusted by the bias voltage of the control voltage. the control voltage has to be positive at all time. 


## *construction*
{: class="subtitle is-lowercase"}

for the final circuit input and output buffering and biasing is needed. the buffering is done with opamps. 

![schematic of the pegel vca](/images/pegel_schematic.svg)


first we design the input stage. the input voltage for the X signal should be 10mV and 20mV for the control voltage.


In [ ]:
#load the kicad schema
directory_path = Path(os.path.abspath('')).resolve()
kicad_netlist_path = directory_path.joinpath('main', 'main.cir')
parser = SpiceParser(path=str(kicad_netlist_path))


In [ ]:
#ringmodulation
ringmod_schema = parser.build_circuit(ground=5)
ringmod_schema.include(spice_library['TL072c'])
ringmod_schema.include(spice_library['AD633'])

ringmod_schema.V('1', '+15V', ringmod_schema.gnd, 'DC 15')
ringmod_schema.V('2', '-15V', ringmod_schema.gnd, 'DC -15')
ringmod_schema.V('3', 'IN_Xa', ringmod_schema.gnd, 'DC 5V AC 5V SIN(0V 5V 1k)')
ringmod_schema.V('4', 'IN_Ya', ringmod_schema.gnd, 'DC 5V AC 5V SIN(0V 5V 100)')
ringmod_schema.V('5', 'IN_Za', ringmod_schema.gnd, 'DC 0V AC 0V')

#simulator = ringmod_schema.simulator(temperature=25, nominal_temperature=25)
#analysis_ringmod = simulator.transient(step_time=10@u_us, end_time=20@u_ms)

In [ ]:
#amplitude modulation
am_schema = parser.build_circuit(ground=5)
am_schema.include(spice_library['TL072c'])
am_schema.include(spice_library['AD633'])

am_schema.V('1', '+15V', am_schema.gnd, 'DC 15')
am_schema.V('2', '-15V', am_schema.gnd, 'DC -15')
am_schema.V('3', 'IN_Xa', am_schema.gnd, 'DC 10V AC 5V SIN(0V 5V 1k)')
am_schema.V('4', 'IN_Ya', am_schema.gnd, 'DC 5V AC 5V SIN(0V 5V 100)')
am_schema.V('5', 'IN_Za', am_schema.gnd, 'DC 5V AC 5V SIN(0V 2.5V 1k)')

#simulator = am_schema.simulator(temperature=25, nominal_temperature=25)
#analysis_am = simulator.transient(step_time=10@u_us, end_time=20@u_ms)

In [ ]:
#vca
vca_schema = parser.build_circuit(ground=5)
vca_schema.include(spice_library['TL072c'])
vca_schema.include(spice_library['AD633'])

vca_schema.V('1', '+15V', vca_schema.gnd, 'DC 15')
vca_schema.V('2', '-15V', vca_schema.gnd, 'DC -15')
vca_schema.V('3', 'IN_Xa', vca_schema.gnd, 'DC 10V AC 5V SIN(0V 5V 1k)')
vca_schema.V('4', 'IN_Ya', vca_schema.gnd, 'DC 5V AC 5V PULSE(0V 10V 1m 1m 15m 1m)')
vca_schema.V('5', 'IN_Za', vca_schema.gnd, 'DC 0V AC 0V')

#simulator = vca_schema.simulator(temperature=25, nominal_temperature=25)
#analysis_vca = simulator.transient(step_time=10@u_us, end_time=20@u_ms)

In [ ]:
unittest.main(argv=[''], verbosity=0, exit=False)

## *references*
{: class="is-subtitle is-size-2 is-lowercase is-italic"}

- [VCA Techniques Investigated][1] Rod Elliott (ESP)
- [Popular Electronics][2] Keyiing and VCA citcuits for electronic music instruments 
- [VCA-1][3] Thomas Henry CA3080 vca
- [VCA-3][4] René Schmitz differential pair vca
- [#223][5]: Basics of the Gilbert Cell | Analog Multiplier | Mixer | Modulator
- [#224][6]: AM & DSB-SC Modulation with the Gilbert Cell
- [Analog Devices][8]: Analog multiplier application guide 
- [AD633][9]: Datasheet 


[1]: https://sound-au.com/articles/vca-techniques.html
[2]: https://tinaja.com/glib/pop_elec/mus_keying_vca_1+2_75.pdf
[3]: https://www.birthofasynth.com/Thomas_Henry/Pages/VCA-1.html
[4]: https://www.schmitzbits.de/vca3.png
[5]: https://www.youtube.com/watch?v=7nmmb0pqTU0&t=2s
[6]: https://www.youtube.com/watch?v=38OQub2Vi2Q
[7]: http://www.ecircuitcenter.com/Circuits/BJT_Diffamp1/BJT_Diffamp1.htm
[8]: https://www.analog.com/media/en/training-seminars/design-handbooks/ADI_Multiplier_Applications_Guide.pdf
[9]: https://www.analog.com/media/en/technical-documentation/data-sheets/AD633.pdf



